<H2>View all basic combination definitions</H2>

In [12]:
import pandas as pd
import numpy as np
import sys; sys.path.append('../') # Locate the helpers file
from LPI_22_0 import *
lusas = get_lusas_modeller()
if not lusas.existsDatabase():
    Exception("A model must be open before running this code")

db = lusas.database() # Reference to the database for convenience


# Read in the combination definitions from the spreadsheet
import os
import_dir = os.path.expanduser("~/Desktop")
file_path = rf"{import_dir}\Delete\combs.xlsx"

create_missing_loadcases = True

In [13]:
def get_loadcase_name(loadcase:str) -> str:
    split = loadcase.split(":")
    # id = int(split[0]) if len(split)>0 else -1
    name = split[1] if len(split)>0 else split[0]
    return name

Basic combinations

In [ ]:
df_basic = pd.read_excel(file_path, sheet_name="Basic Combinations")

for col, row in df_basic.iterrows():
    name = get_loadcase_name(row["Combination"])
    loadcases = []
    factors = []

    for col in df_basic.columns[1:]:
        if isinstance(row.loc[col], str):
            print ("Unknown value ", row.loc[col])
            continue
        if not np.isnan(row.loc[col]):
            factor = float(row.loc[col])
            loadcases.append(col)
            factors.append(factor)


    if(len(loadcases)>0):
        print("Creating Combination : ", name)
        print(list(zip(loadcases, factors)))

        loadcase_names = [get_loadcase_name(loadcase) for loadcase in loadcases]
        
        if db.existsLoadset(name) and db.getLoadset(name).getTypeCode() != 2:
            print(f"Error - Loadcase '{name}' already exists but is not a basic combination")
            continue


        bc = db.createCombinationBasic(name).removeEntries()

        for i in range(len(loadcase_names)):
            
            if not db.existsLoadset(loadcase_names[i]):
                if create_missing_loadcases:
                    db.createLoadcase(loadcase_names[i])
                else:
                    print(f"Warning - Loadcase '{loadcase_names[i]}' does not exist and was not included in combination '{name}'")
                    continue

            bc.addEntry(factors[i], loadcase_names[i])


Creating Combination :  Combination 1
[('1:Loadcase 1', 1.0), ('2:Loadcase 2', 1.0), ('3:Loadcase 3', 1.0)]
Creating Combination :  Combination 2
[('6:Loadcase 6', 1.0), ('7:Loadcase 7', 1.0)]
Creating Combination :  Combination 3
[('1:Loadcase 1', 22.0)]
